In [1]:
datos_entrenamiento = [
    [1, 1, 0, 1, 'Infección de garganta'],
    [0, 0, 1, 0, 'Alergia'],
    [0, 0, 1, 1, 'Resfriado'],
    [0, 0, 1, 0, 'Alergia'],
    [1, 1, 1, 1, 'Infección de garganta'],
    [0, 0, 1, 0, 'Resfriado'],
    [1, 0, 1, 1, 'Resfriado'],
    [1, 1, 0, 1, 'Infección de garganta'],
    [0, 0, 1, 1, 'Resfriado'],
]
#datos recopilados de la matriz de la matriz de decisiones

In [2]:
#Síntomas y condicionales de nuestra matriz
etiquetas = ["Fiebre", "Dolor de garganta", "Congestión", "Dolor de cabeza", "Diagnóstico"]

In [3]:
def unicos_valores(filas, columnas): return set([fila[columnas] for fila in filas])

In [4]:
unicos_valores(datos_entrenamiento, 4)

{'Alergia', 'Infección de garganta', 'Resfriado'}

In [5]:
def class_contar(filas):
    contar = {}
    for fila in filas:
        contenido = fila[-1]
        if contenido not in contar:
            contar[contenido] = 0
        contar[contenido] += 1
    return contar

In [6]:
#contar etiquetas de los datos de entrenamiento
class_contar(datos_entrenamiento)

{'Infección de garganta': 3, 'Alergia': 2, 'Resfriado': 4}

In [7]:
def es_numerico(valor): return isinstance(valor, int) or isinstance(valor, float)

In [12]:
es_numerico(datos_entrenamiento[0][1])

True

In [13]:
class Evaluar:

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def comparar(self, example):
        val = example[self.column]
        if es_numerico(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if es_numerico(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            etiquetas[self.column], condition, str(self.value))

In [24]:
Evaluar(1, 1)

Is Congestión >= 1?

In [25]:
q = Evaluar(1, 1)
q

Is Congestión >= 1?

In [27]:
example = datos_entrenamiento[4]
q.comparar(example)

True

In [28]:
def particionar(filas, evaluar):
    filas_acertadas, filas_erradas = [], []
    for fila in filas:
        if evaluar.comparar(fila):
            filas_acertadas.append(fila)
        else:
            filas_erradas.append(fila)
    return filas_acertadas, filas_erradas

In [35]:
filas_acertadas, filas_erradas = particionar(datos_entrenamiento, Evaluar(4, 'Alergia')) 
filas_acertadas

[[0, 0, 1, 0, 'Alergia'], [0, 0, 1, 0, 'Alergia']]

In [36]:
filas_erradas

[[1, 1, 0, 1, 'Infección de garganta'],
 [0, 0, 1, 1, 'Resfriado'],
 [1, 1, 1, 1, 'Infección de garganta'],
 [0, 0, 1, 0, 'Resfriado'],
 [1, 0, 1, 1, 'Resfriado'],
 [1, 1, 0, 1, 'Infección de garganta'],
 [0, 0, 1, 1, 'Resfriado']]

In [37]:
def incertidumbre(filas):
    conteo = class_contar(filas)
    impureza = 1
    for datos in conteo:
        prob_de_datos = conteo[datos] / float(len(filas))
        impureza -= prob_de_datos**2
    return impureza

In [38]:
incertidumbre_actual = incertidumbre(datos_entrenamiento)
incertidumbre_actual

0.6419753086419753

In [39]:
def informacion_seguimiento(izquierda, derecha, incertidumbre_actual):
    p = float(len(izquierda)) / (len(izquierda) + len(derecha))
    return incertidumbre_actual - p * incertidumbre(izquierda) - (1 - p) * incertidumbre(derecha)


In [52]:
filas_acertadas, filas_erradas = particionar(datos_entrenamiento, Evaluar(1, 1)) 
informacion_seguimiento(filas_acertadas, filas_erradas, incertidumbre_actual)

0.34567901234567894

In [53]:
def encontrar_mejor_ruta(filas):
    
    mejor_seguimiento = 0
    mejor_evaluacion = None
    incertidumbre_actual = incertidumbre(filas)
    numero_columnas = len(filas[0]) - 1

    for columnas in range(numero_columnas):

        valores = set([fila[columnas] for fila in filas])

        for valor in valores:

            evaluar = Evaluar(columnas, valor)
            filas_acertadas, filas_erradas = particionar(filas, evaluar)

            if len(filas_acertadas) == 0 or len(filas_erradas) == 0:
                continue

            info = informacion_seguimiento(filas_acertadas, filas_erradas, incertidumbre_actual)

            if info >= mejor_seguimiento:
                mejor_seguimiento, mejor_evaluacion = info, evaluar

    return mejor_seguimiento, mejor_evaluacion

In [54]:
mejor_seguimiento, mejor_evaluacion = encontrar_mejor_ruta(datos_entrenamiento)
mejor_seguimiento

0.34567901234567894

In [51]:
mejor_evaluacion

Is Dolor de garganta >= 1?

In [55]:
class clasificacion:
   
    def __init__(self, filas):
        self.prediccion = class_contar(filas)

In [56]:
class decidir_nodo:

    def __init__(self, evaluar, rama_correcta, rama_incorrecta):
        self.evaluar = evaluar
        self.rama_correcta = rama_correcta
        self.rama_incorrecta = rama_incorrecta

In [57]:
def construir_arbol(filas):

    info, evaluar = encontrar_mejor_ruta(filas)

    if info == 0:
        return clasificacion(filas)

    filas_acertadas, filas_erradas = particionar(filas, evaluar)
    rama_correcta = construir_arbol(filas_acertadas)
    rama_incorrecta = construir_arbol(filas_erradas)

    return decidir_nodo(evaluar, rama_correcta, rama_incorrecta)

In [58]:
def imprimir_arbol(nodo, espacio=""):

    if isinstance(nodo, clasificacion):
        print (espacio + "Predict", nodo.prediccion)
        return

    print (espacio + str(nodo.evaluar))

    print (espacio + '--> True:')
    imprimir_arbol(nodo.rama_correcta, espacio + "  ")

    print (espacio + '--> False:')
    imprimir_arbol(nodo.rama_incorrecta, espacio + "  ")

In [59]:
mi_arbol = construir_arbol(datos_entrenamiento)

In [60]:
imprimir_arbol(mi_arbol)

Is Dolor de garganta >= 1?
--> True:
  Predict {'Infección de garganta': 3}
--> False:
  Is Dolor de cabeza >= 1?
  --> True:
    Predict {'Resfriado': 3}
  --> False:
    Predict {'Alergia': 2, 'Resfriado': 1}


In [61]:
def clasificar(fila, nodo):

    if isinstance(nodo, clasificacion):
        return nodo.prediccion

    if nodo.evaluar.comparar(fila):
        return clasificar(fila, nodo.rama_correcta)
    else:
        return clasificar(fila, nodo.rama_incorrecta)

In [64]:
clasificar(datos_entrenamiento[0], mi_arbol)

{'Infección de garganta': 3}

In [65]:
def imprimir_clasificacion(conteo):
    total = sum(conteo.values()) * 1.0
    probabilidades = {}
    for datos in conteo.keys():
        probabilidades[datos] = str(int(conteo[datos] / total * 100)) + "%"
    return probabilidades

In [67]:
imprimir_clasificacion(clasificar(datos_entrenamiento[1], mi_arbol))

{'Alergia': '66%', 'Resfriado': '33%'}

In [68]:
evaluar_datos = [
    [1, 0, 1, 1, '¿?'],
    [0, 0, 1, 0, '¿?'],
]

In [69]:
for fila in evaluar_datos:
    print ("Diagnóstico: %s" %
           (imprimir_clasificacion(clasificar(fila, mi_arbol))))

Diagnóstico: {'Resfriado': '100%'}
Diagnóstico: {'Alergia': '66%', 'Resfriado': '33%'}
